In [1]:
import gym
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import deque, namedtuple
import random
import numpy as np
import pickle

Global valriables

In [14]:
GAMMA = 0.99
AVG_OVER = 5
LR = 0.001
BUFFER_SIZE = 100
BATCH_SIZE = 64
UPDATE_EVERY = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
env_cart_pole = gym.make('CartPole-v1')
obs, info = env_cart_pole.reset(seed = 0)
#print(f'Observation space: {obs}, Info: {info}')

env_acro_bot = gym.make('Acrobot-v1')
obs, info = env_acro_bot.reset(seed = 0)
#print(f'Observation space: {obs}, Info: {info}')


In [16]:
def print_env_details(env):
    print("Env name: ", str(env.env)[34:-3])
    print(f'Observation space: {env.observation_space.shape[0]}')
    print(f'Action space: {env.action_space.n}')

In [17]:
print_env_details(env_cart_pole)
print_env_details(env_acro_bot)

Env name:  CartPoleEnv<CartPole-v1>
Observation space: 4
Action space: 2
Env name:  AcrobotEnv<Acrobot-v1>
Observation space: 6
Action space: 3


# Dueling-DQN

In [18]:
class QNetwork1(nn.Module):
    def __init__(self, state_size, action_size, seed, fc1_units=128, fc2_units=64, activation = F.relu):
        super(QNetwork1, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc_v_1 = nn.Linear(fc2_units, 32)
        self.fc_v_2 = nn.Linear(32, 1)
        self.fc_a_1 = nn.Linear(fc2_units, 32)
        self.fc_a_2 = nn.Linear(32, action_size)
        self.activation = activation

    def forward(self, state):
        x = self.activation(self.fc1(state))
        x = self.activation(self.fc2(x))
        v = self.activation(self.fc_v_1(x))
        v = self.fc_v_2(v)
        a = self.activation(self.fc_a_1(x))
        a = self.fc_a_2(a)
        q = v + (a - a.mean())
        return q

In [19]:
model = QNetwork1(4, 2, 0)
model

QNetwork1(
  (fc1): Linear(in_features=4, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc_v_1): Linear(in_features=64, out_features=32, bias=True)
  (fc_v_2): Linear(in_features=32, out_features=1, bias=True)
  (fc_a_1): Linear(in_features=64, out_features=32, bias=True)
  (fc_a_2): Linear(in_features=32, out_features=2, bias=True)
)

In [20]:
import torchviz

# Create a dummy input tensor
state_space = 4
dummy_input = torch.randn(1, state_space)

# Generate the computational graph
graph = torchviz.make_dot(model(dummy_input), params=dict(model.named_parameters()))

# Save the graph as a PDF file
graph.render("computational_graph")


'computational_graph.pdf'

In [21]:
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        return (states, actions, rewards, next_states, dones)
    
    def __len__(self):
        return len(self.memory)
    

In [27]:
class DDQN_Agent():
    def __init__(self, state_size, action_size, seed=42):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        self.qnetwork_local = QNetwork1(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork1(state_size, action_size, seed).to(device)
        self.optimizer = torch.optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)
        
        if(len(self.memory) > BATCH_SIZE):
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)
        
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())

    def act(self, state, eps=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        
    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        Q_expected = self.qnetwork_local(states).gather(1, actions.long())

        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        


In [42]:
def dqn_train(agent,env, n_episodes=10000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):

    scores_window = deque(maxlen=100)
    ''' last 100 scores for checking if the avg is more than 195 '''
    scores = []
    eps = eps_start
    ''' initialize epsilon '''
    for i_episode in range(1, n_episodes+1):
        state = env.reset()[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores.append(score)
        scores_window.append(score)

        eps = max(eps_end, eps_decay*eps)
        ''' decrease epsilon '''

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")

        if i_episode % 100 == 0:
           print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=195.0:
           print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
           
    return True, scores

In [35]:
def avg(l):
    max_len = 0
    for i in l:
        max_len = max(max_len, len(i))
    
    ans = [0] * max_len
    for i in range(max_len):
        sums = 0
        count = 0
        for list_each in l:
            if i < len(list_each):
                count+=1
                sums += list_each[i]
        ans[i] = sums / count
    return ans

In [53]:
def run_agent(env,typee=1,num_epsisodes=1000):
    state_space = env.observation_space.shape[0]
    action_space = env.action_space.n

    env_name = None
    if(state_space == 4):
        print("CartPole-v1")
        env_name = "CartPole-v1"
    else:
        print("Acrobot-v1")
        env_name = "Acrobot-v1"

    total_scores = []
    for i in range(AVG_OVER):
        dualing_dqn_agent = DDQN_Agent(state_space, action_space)
        _, s = dqn_train(dualing_dqn_agent, env, n_episodes = num_epsisodes)
        total_scores.append(s)
        
        pickle.dump(dualing_dqn_agent.qnetwork_local.state_dict(), open(f"learned_agents/local_qnetwork_{env_name}_{typee}_{i+1}.pkl", "wb"))
        pickle.dump(s, open(f"Checkpoints/total_score_{env_name}_{typee}_{i+1}.pkl", "wb"))
    avg_scores = avg(total_scores)

    return avg_scores    

### Type-1 Update rule

\begin{align*}
  Q(s,a;\theta) = V(s;\theta) + (A(s,a;\theta) - \frac{1}{|A|}\sum_{a'\epsilon|A|}A(s,a';\theta))
\end{align*}

In [44]:
def forward(self, state):
        x = self.activation(self.fc1(state))
        x = self.activation(self.fc2(x))
        v = self.activation(self.fc_v_1(x))
        v = self.fc_v_2(v)
        a = self.activation(self.fc_a_1(x))
        a = self.fc_a_2(a)
        q = v + (a - a.mean())
        return q
QNetwork1.forward = forward


In [55]:
scores_1_1 = run_agent(env_cart_pole, typee=1, num_epsisodes=500)

CartPole-v1
Episode 100	Average Score: 37.12
Episode 200	Average Score: 101.62
Episode 285	Average Score: 197.62
Environment solved in 285 episodes!	Average Score: 197.62
Episode 286	Average Score: 200.96
Environment solved in 286 episodes!	Average Score: 200.96
Episode 287	Average Score: 199.76
Environment solved in 287 episodes!	Average Score: 199.76
Episode 288	Average Score: 199.07
Environment solved in 288 episodes!	Average Score: 199.07
Episode 289	Average Score: 201.15
Environment solved in 289 episodes!	Average Score: 201.15
Episode 290	Average Score: 206.89
Environment solved in 290 episodes!	Average Score: 206.89
Episode 291	Average Score: 208.93
Environment solved in 291 episodes!	Average Score: 208.93
Episode 292	Average Score: 211.19
Environment solved in 292 episodes!	Average Score: 211.19
Episode 293	Average Score: 212.35
Environment solved in 293 episodes!	Average Score: 212.35
Episode 294	Average Score: 220.52
Environment solved in 294 episodes!	Average Score: 220.52
E

In [45]:
# scores_1_2 = run_agent(env_acro_bot, typee=1, visualize = False)

### Type-2 Update Rule

\begin{align*}
  Q(s,a;\theta) = V(s;\theta) + (A(s,a;\theta) - \max_{a'\epsilon|A|}\sum_{a'\epsilon|A|}A(s,a';\theta))
\end{align*}

In [27]:
def forward(self, state):
        x = self.activation(self.fc1(state))
        x = self.activation(self.fc2(x))
        v = self.activation(self.fc_v_1(x))
        v = self.fc_v_2(v)
        a = self.activation(self.fc_a_1(x))
        a = self.fc_a_2(a)
        q = v + (a - a.max())
        return q
QNetwork1.forward = forward


In [56]:
score_2_1 = run_agent(env_cart_pole, typee=2, num_epsisodes=500)

CartPole-v1
Episode 3	Average Score: 23.33

Episode 100	Average Score: 36.74
Episode 200	Average Score: 122.02
Episode 278	Average Score: 195.18
Environment solved in 278 episodes!	Average Score: 195.18
Episode 281	Average Score: 195.57
Environment solved in 281 episodes!	Average Score: 195.57
Episode 282	Average Score: 197.20
Environment solved in 282 episodes!	Average Score: 197.20
Episode 283	Average Score: 195.33
Environment solved in 283 episodes!	Average Score: 195.33
Episode 286	Average Score: 196.39
Environment solved in 286 episodes!	Average Score: 196.39
Episode 287	Average Score: 206.20
Environment solved in 287 episodes!	Average Score: 206.20
Episode 288	Average Score: 209.03
Environment solved in 288 episodes!	Average Score: 209.03
Episode 289	Average Score: 213.66
Environment solved in 289 episodes!	Average Score: 213.66
Episode 290	Average Score: 217.68
Environment solved in 290 episodes!	Average Score: 217.68
Episode 291	Average Score: 215.84
Environment solved in 291 episodes!	Average Score: 215.84
Episode 292	A

In [46]:
# score_2_2 = run_agent(env_acro_bot, visualize = False)

# Monte-Carlo REINFORCE

### W/O Baseline

### W/ Baseline